# Multi-Board Quantum Cluster

This notebook demonstrates `QuantumCluster` for distributing quantum circuits across multiple RFSoC boards, and `QuantumCollective` for MPI-style data exchange between boards.

In simulation mode, each "board" runs its own simulation backend. On real hardware, each board would be a separate RFSoC FPGA connected over the network.

In [ ]:
from pynq_quantum import QuantumCircuit
from pynq_quantum.cluster import QuantumCluster

# Create a 2-board cluster using simulation backends
cluster = QuantumCluster(
    hosts=["board-0", "board-1"],
    backend="simulation",
)
cluster.connect()
cluster.sync_clocks()

print(f"Boards: {cluster.num_boards}")
print(f"Total qubits: {cluster.total_qubits}")
for i, board in enumerate(cluster.boards):
    print(f"  Board {i} ({board.host}): connected={board.connected}, qubits={board.num_qubits}")

# Build a 4-qubit circuit and distribute across 2 boards
circ = QuantumCircuit(4, name="distributed")
circ.h(0).cnot(0, 1)      # Entangle qubits 0-1 (board 0)
circ.h(2).cnot(2, 3)      # Entangle qubits 2-3 (board 1)
circ.measure_all()

# Run distributed -- each board handles its partition
results = cluster.run(circ, shots=1000)
for i, r in enumerate(results):
    print(f"Board {i} results: {r.counts}")

cluster.disconnect()

In [ ]:
import numpy as np
from pynq_quantum.collective import QuantumCollective, ReduceOp

# Collective operations for cross-board data exchange
cluster = QuantumCluster(
    hosts=["board-0", "board-1", "board-2"],
    backend="simulation",
)
cluster.connect()

collective = QuantumCollective(cluster)
collective.init_accl()  # Uses software emulation without ACCL hardware

# Broadcast measurement data from board 0 to all boards
data = np.array([0.95, 0.05])  # Probabilities from board 0
broadcast_result = collective.broadcast(data, root=0)
print(f"Broadcast result: {broadcast_result}")

# Allreduce: sum measurement counts across boards
local_data = np.array([250, 750])  # Local counts from this board
reduced = collective.allreduce(local_data, op=ReduceOp.SUM)
print(f"Allreduce (SUM): {reduced}")

# Merge bitstring counts from all boards
local_counts = {"00": 480, "11": 520}
merged = collective.merge_counts(local_counts)
print(f"Merged counts: {merged}")

# Barrier synchronization
collective.barrier()
print("All boards synchronized.")

cluster.disconnect()